In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Hackathon/IIT Madras/Permanent/all_matches.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df=df.drop(labels='season',axis=1)

In [ ]:
df['total_runs']=df.runs_off_bat + df.extras

In [ ]:
df['total_runs_cum'] = df.groupby(['match_id','innings'])['total_runs'].cumsum()

In [ ]:
df=df.drop(labels=['striker', 'non_striker', 'bowler', 'wides','noballs', 'byes', 'legbyes', 'penalty', 'wicket_type','player_dismissed', 'other_wicket_type', 'other_player_dismissed' ,'total_runs','runs_off_bat','extras'],axis=1)

In [ ]:
consistent_team=['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Mumbai Indians','Sunrisers Hyderabad','Delhi Capitals']

In [ ]:
df = df[(df['batting_team'].isin(consistent_team)) & (df['bowling_team'].isin(consistent_team))]

In [ ]:
#df =df[df['ball']<= 5.0]

In [ ]:
df=df[df['innings']<=2]

In [ ]:
df=df.drop(labels=['match_id'],axis=1)

In [ ]:
df=df.drop(labels=['venue'],axis=1)

In [ ]:
df

In [ ]:
df.batting_team.unique()

In [ ]:
team_encoder = LabelEncoder()
df['batting_team'] = team_encoder.fit_transform(df['batting_team'])
df['bowling_team'] = team_encoder.fit_transform(df['bowling_team'])

In [ ]:
df

In [ ]:
df['start_date'] = df['start_date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

In [ ]:
X_train = df.drop(labels='total_runs_cum', axis=1)[df['start_date'].dt.year <= 2016]
X_test =  df.drop(labels='total_runs_cum', axis=1)[df['start_date'].dt.year >= 2017]

y_train =df[df['start_date'].dt.year <= 2016]['total_runs_cum'].values
y_test = df[df['start_date'].dt.year >= 2017]['total_runs_cum'].values

In [ ]:
X_train.drop(labels='start_date',axis=1,inplace=True)
X_test.drop(labels='start_date',axis=1,inplace=True)

In [ ]:
X_train

In [ ]:
rfr=RandomForestRegressor()
rfr.fit(X_train,y_train)
pred=rfr.predict(X_test)
print(r2_score(y_test,pred))

0.8637545448229436


In [ ]:
joblib.dump(rfr,'/content/drive/MyDrive/Hackathon/IIT Madras/Permanent/regressor_model.joblib')
joblib.dump(team_encoder,'/content/drive/MyDrive/Hackathon/IIT Madras/Permanent//team_encoder.joblib')

['/content/drive/MyDrive/Hackathon/IIT Madras/Permanent//team_encoder.joblib']